In [55]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import time

In [56]:
import ipynb.fs.full.TrainTest as TrainTest

nperseg = 2
p = 30 # want this to be high
a = [2, 3]
w = 4  # want this to be low
cw = 9 # want this to be high

path = './data/created/nperseg' + str(nperseg) + '/'
file = 'p' + str(p) + '_a' + str(a) + '_w' + str(w) + '_cw' + str(cw)
ext = '.npy'
data = np.load(path + file + ext)

x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

Do RandomizedSearchCV

In [57]:
def getBestParams(clf, param_options):
    clf_grid = GridSearchCV(estimator = clf, param_grid = param_options, cv = 3, verbose=1, n_jobs = -1)
    clf_grid.fit(x_train, y_train)
    return clf_grid.best_params_

In [58]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

# options for nearest neighbors
param_options = {
    'n_neighbors' : range(1, 6),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto'],
    'leaf_size' : [10, 20, 30, 40, 50],
    'p' : [1, 2, 3, 4],
    'metric' : ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
#     'metric_params' : [],
#     'n_jobs' : []
}

best = getBestParams(KNeighborsClassifier(), param_options)
knn = KNeighborsClassifier(**best)

avg_acc = 0
avg_prec = 0
avg_f1 = 0
avg_train_time = 0
avg_test_time = 0

num = 10
for i in range(num):
    train_start = time.clock()
    knn.fit(x_train, y_train)
    train_end = time.clock()
    
    test_start = time.clock()
    y_pred = knn.predict(x_test)
    test_end = time.clock()
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)
    avg_train_time += (train_end - train_start)
    avg_test_time += (test_end - test_start)

avg_acc /= num
avg_prec /= num
avg_f1 /= num
avg_train_time /= num
avg_test_time /= num

with open('./results/knn/' + file + '.txt', 'a+') as f:
    f.write('(nperseg = {}, best = {})\n'.format(nperseg, best))
    f.write('trials         : {}\n'.format(num))
    f.write('avg acc        : {}\n'.format(avg_acc))
    f.write('avg prec       : {}\n'.format(avg_prec))
    f.write('avg f1         : {}\n'.format(avg_f1))
    f.write('avg_train_time : {}\n'.format(avg_train_time))
    f.write('avg_test_time  : {}\n'.format(avg_test_time))
    f.write('-----\n\n')

Fitting 3 folds for each of 800 candidates, totalling 2400 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:  8.2min finished
